In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from kan import KAN  # Импортируем KAN из pykan
import numpy as np

In [2]:
# Устройство: если есть GPU, используем его, иначе CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем: {device} — надеюсь, это не старый калькулятор!")

Используем: cpu — надеюсь, это не старый калькулятор!


In [3]:
# 1. Загружаем MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
# 2. Настраиваем KAN
# Вход: 784 (28x28 пикселей), скрытый слой: 64 нейрона, выход: 10 классов (0-9)
model = KAN([784, 64, 10], grid=5, k=3, device=device).to(device)

checkpoint directory created: ./model
saving model version 0.0


In [6]:
# 3. Оптимизатор и функция потерь
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Адам — лучший друг нейронок
criterion = nn.CrossEntropyLoss()  # Классика для классификации

In [7]:
# 4. Функция обучения
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(-1, 784)  # Разворачиваем картинку в вектор
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f'Эпоха {epoch}, батч {batch_idx}: Потери = {loss.item():.4f} — KAN старается!')
    print(f'Эпоха {epoch} завершена! Средние потери: {total_loss / len(train_loader):.4f}')


In [8]:
# 5. Функция теста
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(-1, 784)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100. * correct / total
    print(f'Точность на тесте: {accuracy:.2f}% — KAN либо гений, либо притворяется!')

In [9]:
# 6. Запускаем обучение — полный вперёд!
num_epochs = 5  # 5 эпох хватит, чтобы не утомить KAN
for epoch in range(1, num_epochs + 1):
    train(model, train_loader, optimizer, criterion, epoch)
    test(model, test_loader)


Эпоха 1, батч 0: Потери = 2.2996 — KAN старается!


KeyboardInterrupt: 

In [ ]:
# 7. Финальный аккорд
print("Готово! KAN классифицировал MNIST, как будто это теорема из учебника!")

# Вывод символической формулы
symbolic_formula, variables = model.symbolic_formula()
print("\nСимволическая формула для первого выхода (класс 0):")
print(symbolic_formula[0])  # Первый выход (для класса 0)
print("\nПеременные (x_1, x_2, ... — это пиксели):")
print(variables[:10], "... и ещё куча других!")

# Пример интерпретации (опционально)
print("\nKAN говорит: 'Вот как я вижу мир!'")